In [199]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.style.use('fivethirtyeight')

from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Read Data

In [261]:
data_path = '../../input/'
df_reg_detailed_results = pd.read_csv(data_path + 'RegularSeasonDetailedResults.csv')
df_tourney_detailed_results = pd.read_csv(data_path + 'NCAATourneyDetailedResults.csv')
df_seeds = pd.read_csv(data_path + 'NCAATourneySeeds.csv')
df_seasons = pd.read_csv(data_path + 'Seasons.csv')
df_teams = pd.read_csv(data_path + 'Teams.csv')

In [269]:
df_reg_detailed_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [272]:
df_tourney_detailed_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,134,1421,92,1411,84,N,1,32,69,...,31,14,31,17,28,16,15,5,0,22
1,2003,136,1112,80,1436,51,N,0,31,66,...,16,7,7,8,26,12,17,10,3,15
2,2003,136,1113,84,1272,71,N,0,31,59,...,28,14,21,20,22,11,12,2,5,18
3,2003,136,1141,79,1166,73,N,0,29,53,...,17,12,17,14,17,20,21,6,6,21
4,2003,136,1143,76,1301,74,N,1,27,64,...,21,15,20,10,26,16,14,5,8,19


In [263]:
df_seeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [204]:
df_seasons.head()

,Season,DayZero,RegionW,RegionX,RegionY,RegionZ
0,1985,10/29/1984,East,West,Midwest,Southeast
1,1986,10/28/1985,East,Midwest,Southeast,West
2,1987,10/27/1986,East,Southeast,Midwest,West
3,1988,11/2/1987,East,Midwest,Southeast,West
4,1989,10/31/1988,East,West,Midwest,Southeast


In [257]:
df_teams.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2018
1,1102,Air Force,1985,2018
2,1103,Akron,1985,2018
3,1104,Alabama,1985,2018
4,1105,Alabama A&M,2000,2018


# Team Statistics

In [258]:
df_team_stats = df_teams.copy()

def getWinCountBySeason(season):
    df_win_IDs = pd.DataFrame()
    df_win_IDs['Season'] = df_detailed_results.Season
    df_win_IDs['WTeamID'] = df_detailed_results.WTeamID
    df_win_IDs = df_win_IDs[df_win_IDs.Season == season].drop(labels=['Season'], axis=1)
    
    df_win_count = df_win_IDs.apply(pd.value_counts)
    df_win_count = df_win_count.rename(columns={'WTeamID': 'WCount'})
    
    df_win_tbl = pd.DataFrame()
    df_win_tbl['TeamID'] = list(df_win_count.index)
    df_win_tbl[str(season) + ' WCount'] = df_win_count.WCount.values
    return df_win_tbl

def getLossCountBySeason(season):
    df_loss_IDs = pd.DataFrame()
    df_loss_IDs['Season'] = df_detailed_results.Season
    df_loss_IDs['WTeamID'] = df_detailed_results.WTeamID
    df_loss_IDs = df_loss_IDs[df_loss_IDs.Season == season].drop(labels=['Season'], axis=1)
    
    df_loss_count = df_loss_IDs.apply(pd.value_counts)
    df_loss_count = df_loss_count.rename(columns={'WTeamID': 'WCount'})
    
    df_loss_tbl = pd.DataFrame()
    df_loss_tbl['TeamID'] = list(df_loss_count.index)
    df_loss_tbl[str(season) + ' WCount'] = df_loss_count.WCount.values
    return df_loss_tbl

getWinCountBySeason(2010)

,TeamID,2010 WCount
0,1181,6
1,1139,5
2,1277,4
3,1452,4
4,1246,3
5,1243,3
6,1397,3
7,1124,3
8,1393,2
9,1388,2


# Seed Feature

In [179]:
def seed2Int(seed):
    s_int = int(seed[1:3])
    return s_int

df_seeds['Seed'] = df_seeds.Seed.apply(seed2Int)
df_seeds.head()

,Season,Seed,TeamID
0,1985,1,1207
1,1985,2,1210
2,1985,3,1228
3,1985,4,1260
4,1985,5,1374


In [198]:
df_win_seeds = df_seeds.rename(columns={'TeamID': 'WTeamID', 'Seed': 'WSeed'})
df_loss_seeds = df_seeds.rename(columns={'TeamID': 'LTeamID', 'Seed': 'LSeed'})
df_concat = pd.merge(left=df_compact_results, right=df_win_seeds, how='left', on=['Season', 'WTeamID'])
df_concat = pd.merge(left=df_concat, right=df_loss_seeds, on=['Season', 'LTeamID'])
df_concat['SeedDiff'] = df_concat.WSeed - df_concat.LSeed
df_concat.head()

,Season,WTeamID,LTeamID,WSeed,LSeed,SeedDiff
0,1985,1116,1234,9,8,1
1,1985,1120,1345,11,6,5
2,1985,1207,1250,1,16,-15
3,1985,1229,1425,9,8,1
4,1985,1242,1325,3,14,-11
